In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dense, Concatenate, TimeDistributed, Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.optimizers import Adam

max_features = 10000
maxlen = 100
embedding_dim =64
latent_dim = 128
batch_size = 64
epochs = 10

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

encoder_inputs = Input(shape=(maxlen,))
encoder_embedding = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=maxlen)(encoder_inputs)
encoder = LSTM(latent_dim, return_sequences=True,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding)
encoder_states = [state_h, state_c]


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [2]:
# 디코더 정의
decoder_inputs = Input(shape=(maxlen, ))
# 임베딩 추가
decoder_embedding = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=maxlen)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# 어텐션
attention = Attention()
attention_output = attention([decoder_lstm_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1)([decoder_lstm_outputs, attention_output])

# 출력 레이어
decoder_dense = TimeDistributed(Dense(max_features, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)


In [3]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

decoder_input_data = np.zeros_like(x_train)
decoder_target_data = np.zeros_like(x_train)

for i, seq in enumerate(x_train):
    decoder_input_data[i, 1:] = seq[:-1]
    decoder_target_data[i, :-1] = seq[1:]

model.fit([x_train, decoder_input_data], np.expand_dims(decoder_target_data, -1), epochs=epochs, validation_split=0.2, batch_size=batch_size)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 64)   │    640,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 100, 64)   │    640,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 100,      │     98,816 │ embedding[0][0]   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 100,      │     98,816 │ embedding_1[0][0… │
│                     │ 128), (None,      │            │ lstm[0][1],       │
│                     │ 128), (None,      │            │ lstm[0][2]        │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 100, 128)  │          0 │ lstm_1[0][0],     │
│ (Attention)         │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 100, 256)  │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 100,       │  2,570,000 │ concatenate[0][0] │
│ (TimeDistributed)   │ 10000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,047,632 (15.44 MB)

 Trainable params: 4,047,632 (15.44 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 441s 1s/step - accuracy: 0.0766 - loss: 6.8526 - val_accuracy: 0.0956 - val_loss: 6.1544
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 517s 2s/step - accuracy: 0.0991 - loss: 6.1023 - val_accuracy: 0.1105 - val_loss: 5.9856
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 430s 1s/step - accuracy: 0.1195 - loss: 5.9137 - val_accuracy: 0.1693 - val_loss: 5.6945
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 452s 1s/step - accuracy: 0.2408 - loss: 5.2629 - val_accuracy: 0.5438 - val_loss: 3.3464
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 489s 2s/step - accuracy: 0.5998 - loss: 2.9086 - val_accuracy: 0.7119 - val_loss: 2.1260
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 539s 2s/step - accuracy: 0.7417 - loss: 1.8914 - val_accuracy: 0.7986 - val_loss: 1.5447
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 554s 2s/step - accuracy: 0.8198 - loss: 1.3449 - val_accuracy: 0.8523 - val_loss: 1.1462
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 552s 2s/step - accuracy: 0.8700 - loss: 0.9874 - val_accu

In [4]:
model.save('imdb_attention_embedding.keras')